In [34]:
# Copyright (c) ContextualFairness contributors.
# Licensed under the MIT License.

# Basic notebook showing the various properties of contextual fairness

In [35]:
import sys

sys.path.append("../")

import pandas as pd
import polars as pl

from contextualfairness.scorer import contextual_fairness_score
from contextualfairness.norms import BinaryClassificationEqualityNorm, RankNorm

### Create fake data

In [36]:
ages = ["young", "young", "old", "young", "old"]
y_true = [1, 0, 0, 1, 0]

data = {
    "income": [50, 80, 30, 100, 30],
    "age": ages,
    "sex": ["male", "female", "male", "female", "male"],
    "y true": y_true,
}

X = pd.DataFrame(data=data)

### Define predictions and probabilities for each prediction

We do not actually train a model for this basic notebook. Rather, we define all the data needed for contextual fairness manually.

The probabilities in `y_pred_probas` denote the probability for each indivual being predicted `1`.

In [37]:
y_pred = [1, 0, 1, 0, 1]
y_pred_probas = [0.5, 0.2, 0.6, 0.4, 0.5]

### Define norms

We use two norms in this scenario: the equality norm for binary classification and a rank norm.
For the rank norm, we need to specify a function that ranks all individuals with respect to this norm.
This function takes an individual `x` as input and returns a value that can be used for ranking.
In this case, we define the function `richer_is_better` that will rank individuals based on their income.

In [38]:
norms = [
    BinaryClassificationEqualityNorm(),
    RankNorm(norm_statement=pl.col("income"), name="richer_is_better"),
]

To calculate the contextual fairnesss for a model and a dataset, we call `contextual_fairness_score` with the norms, the dataset `X`, the predictions `y_pred` and the probabilities `y_pred_probas`. We also specify a weight for each norm. These `weights` are used for weighing the score for each norm.

In [39]:
result = contextual_fairness_score(
    norms=norms,
    data=X.to_dict("list"),
    y_pred=y_pred,
    outcome_scores=y_pred_probas,
    weights=[0.4, 0.6],
)

The `result` of contextual fairness contains a `pandas.DataFrame` that has the same columns as `X` and additional columns for each norm and a column `total` containing the sum of the norms for each individual.

In [40]:
result.df.collect()

income,age,sex,y true,predictions,outcomes,equality,richer_is_better,total
i64,str,str,i64,i64,f64,f64,f64,f64
50,"""young""","""male""",1,1,0.5,0.0,0.06,0.06
80,"""young""","""female""",0,0,0.2,0.2,0.18,0.38
30,"""old""","""male""",0,1,0.6,0.0,0.0,0.0
100,"""young""","""female""",1,0,0.4,0.2,0.18,0.38
30,"""old""","""male""",0,1,0.5,0.0,0.0,0.0


The `result` has a method `total_score` that returns the sum of the values in the `total` column.

In [41]:
result.total_score()

0.8200000000000001

The `result` has a method `group_scores` that returns the contextual fairness score for each group given a list of attributes in `X`. The returned groups are the combinations of all values for the attributes in `X`. Besides the score also the data used for calculating the group score is returned, i.e., the score for each individual in the group.

In [42]:
result.group_scores(["sex", "age"]).collect()

sex,age,total,indices
str,str,f64,list[u32]
"""male""","""old""",0.0,"[2, 4]"
"""male""","""young""",0.06,[0]
"""female""","""old""",0.0,[]
"""female""","""young""",0.76,"[1, 3]"


By setting the `scaled` flag to true, the scores for each group are scaled relative to the size of the group.

In [43]:
result.group_scores(["sex", "age"], scaled=True).collect()

sex,age,total,indices
str,str,f64,list[u32]
"""male""","""old""",0.0,"[2, 4]"
"""male""","""young""",0.111818,[0]
"""female""","""old""",0.0,[]
"""female""","""young""",0.708182,"[1, 3]"
